In [1]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, TimestampType 	



# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('json-aggregator')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
         .getOrCreate())
sc = spark.sparkContext




In [2]:
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:29092") # kafka server      
  .option("subscribe", "patient-data") # topic
  .option("startingOffsets", "earliest") # start from beginning
  .load())



In [3]:
# create schema for patient
mySchema = StructType([
 StructField("id", IntegerType()),
 StructField("nome", StringType()),
 StructField("idade", IntegerType()),
 StructField("sexo", IntegerType()),
 StructField("peso", DoubleType()),
 StructField("altura", IntegerType()),
 StructField("bpm", DoubleType()),
 StructField("pressao", DoubleType()),
 StructField("respiracao", DoubleType()),
 StructField("temperatura", DoubleType()),
 StructField("glicemia", DoubleType()),
 StructField("saturacao_oxigenio", DoubleType()),
 StructField("estado_atividade", IntegerType()),
 StructField("dia_de_semana", IntegerType()),
 StructField("periodo_do_dia", IntegerType()),
 StructField("semana_do_mes", IntegerType()),
 StructField("estacao_do_ano", IntegerType()),
 StructField("passos", IntegerType()),
 StructField("calorias", DoubleType()),
 StructField("distancia", DoubleType()),
 StructField("tempo", DoubleType()),
 StructField("total_sleep_last_24", DoubleType()),
 StructField("deep_sleep_last_24", DoubleType()),
 StructField("light_sleep_last_24", DoubleType()),
 StructField("awake_last_24", DoubleType()),
 StructField("fumante", IntegerType()),
 StructField("genetica", IntegerType()),
 StructField("gestante", IntegerType()),
 StructField("frutas", IntegerType()),
 StructField("vegetais", IntegerType()),
 StructField("alcool", IntegerType()),
 StructField("doenca_coracao", IntegerType()),     
 StructField("avc", IntegerType()),
 StructField("colesterol_alto", IntegerType()), 
 StructField("exercicio", IntegerType()), 
 StructField("timestampstr", TimestampType()),
 StructField("timestamp_epoch", StringType())       
 
])

# extract data and ensure `eventTime` is timestamp
df = (
    df.selectExpr("CAST(value as string)")
      .select(F.from_json(F.col("value"),mySchema).alias("json_value"))
      .selectExpr("json_value.*") # gives us a dataframe with columns (eventTime,temperatura, etc...)
      .select(
          F.expr("CAST(timestampstr as timestamp)").alias("eventTime"),
          F.col("nome"),
          F.col("temperatura"),
          F.col("bpm")
      )
      
)


In [4]:

# when using window you will get a range or value resembling [start,end]. 
# I have chosen the `start` for this example
from pyspark.sql.functions import col, window

windowedAvg = ( 
    df.withWatermark("eventTime", "5 minutes") 
      .groupBy(window(F.col("eventTime"), "5 minutes").alias('eventTimeWindow'), F.col("nome"))
      .agg(F.avg("temperatura").alias("avgtemperature"),F.avg("bpm").alias("avgbpm"))       
      .orderBy(F.col("eventTimeWindow"))
      .select(
          F.col("eventTimeWindow.start").alias("eventTime"),
          F.col("nome"),
          F.col("avgtemperature"),
          F.col("avgbpm")
          
      )
)


In [5]:

# continue with your code to write to your various streams
query = windowedAvg\
        .select(
            F.expr("CAST(eventTime AS STRING)").alias("key"),
            F.expr("'{\"eventTime\":\"' || CAST(eventTime AS STRING) || '\",' || '\"nome\":' || CAST(nome AS STRING) || ',' || '\"avgbpm\":' || CAST(avgbpm AS STRING) || ',' || '\"avgtemp\":' || CAST(avgtemperature AS STRING) || '}'").alias("value")            
        ) \
        .writeStream\
        .outputMode('complete')\
        .format('console')\
        .option('truncate', 'true')\
        .start()



In [6]:

# write on kafka topic avgtemperature
# here i've chosen as an example to use the eventTime as the key and the value to be the avgtemperature
qk = (windowedAvg 
        .select(
            F.expr("CAST(eventTime AS STRING)").alias("key"),
            F.expr("'{\"eventTime\":\"' || CAST(eventTime AS STRING) || '\",' || '\"nome\":\"' || nome || '\",' || '\"avgbpm\":' || CAST(avgbpm AS STRING) || ',' || '\"avgtemp\":' || CAST(avgtemperature AS STRING) || '}'").alias("value")            
        ) 
        .writeStream 
        .format("kafka")
        .option("kafka.bootstrap.servers", "kafka:29092") 
        .option("checkpointLocation", "/home/jovyan/work/json/checkpoint") 
        .option("topic", "avg-data")        
        .outputMode("complete") 
        .start()
        .awaitTermination())
        #query.awaitTermination()

StreamingQueryException: Partition patient-data-0's offset was changed from 201 to 10, some data may have been missed. 
Some data may have been lost because they are not available in Kafka any more; either the
 data was aged out by Kafka or the topic may have been deleted before all the data in the
 topic was processed. If you don't want your streaming query to fail on such cases, set the
 source option "failOnDataLoss" to "false".
    
=== Streaming Query ===
Identifier: [id = 8c7b0016-4ea7-4e8f-bb3f-9c7bbe718911, runId = ef23a8d2-61b2-429c-b2b6-1953f718971b]
Current Committed Offsets: {KafkaV2[Subscribe[patient-data]]: {"patient-data":{"0":201}}}
Current Available Offsets: {KafkaV2[Subscribe[patient-data]]: {"patient-data":{"0":10}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource org.apache.spark.sql.kafka010.KafkaStreamingWrite@76965309
+- Project [cast(eventTime#119 as string) AS key#131, concat(concat(concat(concat(concat(concat(concat(con